# Calculate Interpolation Objects in Batches with Job Arrays

In this notebook we will walk through how to create interpolation objects that 
are constructed from batches of effective parameter data. This data was generated
by batching voltage control vectors and running each batch in parallel on the HPC cluster.

**NOTE:** Utilizing job arrays allows for an order of magnitude speed up when 
calcualting effective parameters.

Example: Serial job took 168 hours = 7 days -> Parallel job with 10 CPUs/workers took 16.8 hours!

First change base working directory and import relevant modules.

In [1]:
import os
from pathlib import Path
path = Path(os.getcwd())

# update base working directory to QuDiPy
if path.stem != 'QuDiPy':
    print(path.parents[1])

    base_dir = path.parents[1]
    os.chdir(base_dir)
else:
    base_dir = path
    
import sys
import numpy as np

from qudipy.system import DotArray

c:\QuDiPy


## Mimic HPC Batch Job Execution

We can test how the parallel computation would take place on the cluster with multiple CPUs, but 
instead perform a local serial calculation using only one CPU. The bash script that is shown later will
define how a python file is called in a job array compared to this toy example.

### Toy 4 Batch Example

We can mimic a HPC job array by calling a python script in a for loop as follows.
The python script requires two inputs:
1) The current batch to evaluate
2) The total number of batches that are to be run (in parallel on the HPC cluster)

In [2]:
batches = 4

In [3]:
for idx in range(batches):
    batch = idx + 1 # too mimic hpc
    %run parallel_batch_example_2QD.py {batch} {batches}


------------------ Calculating batch 1/4 now ------------------

g_factor evaluation: control vector=[0.2 0.1 0.4]:  92%|█████████▏| 11/12 [00:11<00:01,  1.01s/it]                       Warning: number of detected dots is 1 -> different from user-specified value 2. Parameters are not evaluated
g_factor evaluation: control vector=[0.2 0.1 0.4]: 100%|██████████| 12/12 [00:11<00:00,  1.08it/s]
Exchange HL evaluation: control vector=[0.2 0.1 0.4]:  92%|█████████▏| 11/12 [00:00<00:00, 22.12it/s]                       Warning: number of detected dots is 1 -> different from user-specified value 2. Parameters are not evaluated
Exchange HL evaluation: control vector=[0.2 0.1 0.4]: 100%|██████████| 12/12 [00:00<00:00, 22.11it/s]
Exchange HM evaluation: control vector=[0.2 0.1 0.4]:  75%|███████▌  | 9/12 [00:00<00:00, 23.62it/s]                        Warning: number of detected dots is 1 -> different from user-specified value 2. Parameters are not evaluated
Exchange HM evaluation: control vecto

## Construct Effective Parameter Interpolator From Data Set Batches

In [4]:
# define input/output directories
nav_dir = os.path.join('QuDiPy data', 'tutorials')
processed_dir = os.path.join(base_dir, nav_dir,
                        'processed','2QD_processed')
nextnano_dir = os.path.join(base_dir, nav_dir,
                                'nextnano','2QD_dotsep_60nm')

# anticipated number of dots
n_dots = 2      

# define subset of control ranges to perform calcuations
eff_interp_dims = [4,3,4]
ctrl_vals = [np.linspace(0.2, 0.4, eff_interp_dims[0]),
    np.linspace(-0.1, 0.1, eff_interp_dims[1]),
    np.linspace(0.2, 0.4, eff_interp_dims[2])]

# prefix for saved calculated files
file_prefix = 'example_hpc'

dots = DotArray(n_dots, ctrl_ranges=ctrl_vals, calc=False, hpc=[None, batches,'example_hpc_2QD'])
dots.numeric(processed_dir, nextnano_dir, file_prefix)

dots.construct_interpolator()

Note: No effective parameter calcualtions specified. Default = "spin". 

Loading pre-calculated effective parameters: Batch 4 or 4: 100%|██████████| 4/4 [00:00<00:00, 454.10it/s]
Effective parameter interpolator saved as:
	 example_hpc_spin_data_size_[4 3 4]_from_[ 0.2 -0.1  0.2]_to_[0.4 0.1 0.4].pkl
